In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, Conv2D, AveragePooling2D, Flatten
from keras.models import load_model, Model
from keras.models import Sequential
import keras
import numpy as np
from google.colab.patches import cv2_imshow

## Import image

In [ ]:
import cv2
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
feet_path = "/content/gdrive/MyDrive/Dataset_ARM_FEET/200 feets"
hand_path = "/content/gdrive/MyDrive/Dataset_ARM_FEET/200 hands"

In [ ]:
feet_imgs = [cv2.imread(file) for file in glob.glob(feet_path + "/*.jpg")]
hand_imgs = [cv2.imread(file) for file in glob.glob(hand_path + "/*.jpg")]

##1.Create model = vgg16 - softmaxLayer
output dim = (1, 4096)

In [ ]:
vgg16_model = VGG16(weights='imagenet', input_shape=(224,224,3))
model = Sequential()

for layer in vgg16_model.layers[:-1]:
  layer.trainable = False
  model.add(layer)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

##2.Get feature of img by model

In [ ]:
feet_imgsResize = [cv2.resize(img, (224, 224)) for img in feet_imgs]
hand_imgsResize = [cv2.resize(img, (224, 224)) for img in hand_imgs]

In [ ]:
feet_features = [model.predict(img.reshape(1,224,224,3)) for img in feet_imgsResize]
hand_features = [model.predict(img.reshape(1,224,224,3)) for img in hand_imgsResize]

1/1 [==============================] - 1s 604ms/step


In [ ]:
feet_features[0].shape

(1, 4096)

##3.Train/Test Dataset for train SVM

In [ ]:
X_train = [f.reshape(4096) for f in feet_features[:len(feet_features)//10*7] + hand_features[:len(hand_features)//10*7]]
X_test = [f.reshape(4096) for f in feet_features[len(feet_features)//10*7:] + hand_features[len(hand_features)//10*7:]]
Y_train = [0]*(len(feet_features)//10*7) + [1]*(len(hand_features)//10*7)
Y_test = [0]*(len(feet_features) - len(feet_features)//10*7) + [1]*(len(hand_features) - len(hand_features)//10*7)

In [ ]:
print(X_train[0].shape)
print(len(Y_train))
print(len(Y_test))

(4096,)
280
120


##4.Create SVM model

In [ ]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

###4.1.kernel = 'linear'

In [ ]:
clf = svm.SVC(kernel="linear")
clf.fit(X_train, Y_train)

SVC(kernel='linear')

In [ ]:
Y_predict = clf.predict(X_test)

In [ ]:
print(Y_predict)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1]


**Accuracy**

In [ ]:
print('Acc:', accuracy_score(Y_test, Y_predict))

Acc: 0.95


**Confusion matrix**

In [ ]:
confusion_matrix(Y_test, Y_predict)

array([[57,  3],
       [ 3, 57]])

###4.2.kernel = 'rbf'

In [ ]:
clf = svm.SVC(kernel="rbf")
clf.fit(X_train, Y_train)

SVC()

In [ ]:
Y_predict = clf.predict(X_test)

In [ ]:
# Kết quả dự đoán
print(Y_predict)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1]


**Accuracy**

In [ ]:
print('Acc:', accuracy_score(Y_test, Y_predict))

Acc: 0.9333333333333333


**Confusion matrix**

In [ ]:
confusion_matrix(Y_test, Y_predict)

array([[57,  3],
       [ 5, 55]])